In [1]:
import pandas as pd
import numpy as np

In [2]:
INPUT_FILE = '../dataset/flipkart_products_preprocessed_2.csv'

In [3]:
df = pd.read_csv(INPUT_FILE)

In [4]:
df["text"] = (
    df["title"].fillna("") + " " +
    df["description"].fillna("") + " " +
    df["level_combined"].fillna("")
)

In [5]:
features = [
    "price",
    "rating_value",
    "level_2",
    "level_3", 
    "text"
]

X = df[features]
y = df["discount"]

In [6]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from category_encoders.target_encoder import TargetEncoder
from sklearn.metrics import mean_squared_error, r2_score

In [7]:
numeric_features = [
    "price",
    "rating_value"
]

categorical_features = [
    "level_2",
    "level_3"
]

text_features = [
    "text"
]

In [8]:
numeric_transformer = StandardScaler()
categorical_transformer = TargetEncoder()

text_tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=5000
)

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("text_tfidf", text_tfidf, "text")
    ],
    remainder="drop"
)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", XGBRegressor(
        tree_method="hist",
        n_jobs=-1,
        random_state=42
    ))
])

In [12]:
pipeline.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [13]:
y_pred = pipeline.predict(X_test)

In [14]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R^2:", r2)

RMSE: 12.411418896928785
R^2: 0.5866012054490739


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# param_dist = {
#     "model__n_estimators": [300, 500, 800],
#     "model__learning_rate": [0.01, 0.05, 0.1],
#     "model__max_depth": [6, 8, 10],
#     "model__subsample": [0.6, 0.8, 1.0],
#     "model__colsample_bytree": [0.6, 0.8, 1.0],
#     "model__reg_alpha": [0, 0.1, 1],
#     "model__reg_lambda": [1, 3, 5]
# }

In [ ]:
# search = RandomizedSearchCV(
#     pipeline,
#     param_dist,
#     n_iter=20,
#     scoring="neg_root_mean_squared_error",
#     cv=2,
#     n_jobs=-1,
#     verbose=2,
#     random_state=42
# )

In [ ]:
# search.fit(X_train, y_train)

# print(search.best_params_)
# print(-search.best_score_)

In [ ]:
# best_xgboost = search.best_estimator_

# y_pred = best_xgboost.predict(X_test)

# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# r2 = r2_score(y_test, y_pred)

# print("RMSE:", rmse)
# print("R^2:", r2)